# Setup

Download the Task data and evaluation scripts

In [ ]:
!git clone https://github.com/H-TayyarMadabushi/SemEval_2022_Task2-idiomaticity.git

fatal: destination path 'SemEval_2022_Task2-idiomaticity' already exists and is not an empty directory.


Download the “AStitchInLanguageModels” code which we make use of.

In [ ]:
!git clone https://github.com/H-TayyarMadabushi/AStitchInLanguageModels.git

fatal: destination path 'AStitchInLanguageModels' already exists and is not an empty directory.


Download and install an editable version of huggingfaces transformers.

In [ ]:
!git clone https://github.com/huggingface/transformers.git
%cd transformers/
!pip install --editable .
%cd /content/

fatal: destination path 'transformers' already exists and is not an empty directory.
/content/transformers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/transformers
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for transformers (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.28.0.dev0-0.editable-py3-none-any.whl size=35088 sha256=e71be3776f4dff23d06564a95c1352c5b8d9487d2a5e70694c8c9573401af18e
  Stored in directory: /tmp/pip-ephem-wheel-cache-081aqwtm/wheels/1c/6e/db/b90d9f8554f165a9beb90b593fde94e9e60919270aed78efa0
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.28.0.dev0
    Uninstalling transformers-4.28.0.dev0:
      Suc

Required for run_glue ...

In [ ]:
## run_glue needs this.
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Editable install requires runtime restart unless we do this.

In [ ]:
import site
site.main()


In [ ]:
pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Imports and Helper functions

In [ ]:
import os
import csv
import imblearn
from imblearn.under_sampling import RandomUnderSampler
import numpy as np
from pathlib import Path
import pandas as pd

In [ ]:
def load_csv( path, delimiter=',' ) :
  header = None
  data   = list()
  with open( path, encoding='utf-8') as csvfile:
    reader = csv.reader( csvfile, delimiter=delimiter )
    for row in reader :
      if header is None :
        header = row
        continue
      data.append( row )
  return header, data


In [ ]:
def write_csv( data, location ) :
  with open( location, 'w', encoding='utf-8') as csvfile:
    writer = csv.writer( csvfile )
    writer.writerows( data )
  print( "Wrote {}".format( location ) )
  return


The following function creates a submission file from the predictions output by run_glue (the text classification script from huggingface transformers - see below).

Note that we set it up so we can load up results for only one setting.

It requires as input the submission format file, which is available with the data. You can call this after completing each setting to load up results for both settings (see below).


In [ ]:
def insert_to_submission_file( submission_format_file, input_file, prediction_format_file, setting ) :
    submission_header, submission_content = load_csv( submission_format_file )
    input_header     , input_data         = load_csv( input_file             )
    prediction_header, prediction_data    = load_csv( prediction_format_file, '\t' )

    assert len( input_data ) == len( prediction_data )

    ## submission_header ['ID', 'Language', 'Setting', 'Label']
    ## input_header      ['label', 'sentence1' ]
    ## prediction_header ['index', 'prediction']

    prediction_data = list( reversed( prediction_data ) )

    started_insert  = False
    for elem in submission_content :
        if elem[ submission_header.index( 'Setting' ) ] != setting :
            if started_insert :
                if len( prediction_data ) == 0 :
                    break
                else :
                    raise Exception( "Update should to contiguous ... something wrong." )
            continue
        started_insert = True
        elem[ submission_header.index( 'Label' ) ] = prediction_data.pop()[ prediction_header.index( 'prediction' ) ]

    return [ submission_header ] + submission_content

# Pre-process: Create train and dev and evaluation data in required format

In the zero-shot setting, we choose to include the context (the sentences preceding and succeeding the one containing the idioms). We do not add the idiom as an additional feature (in the “second input sentence”).

In the one shot setting, we train the model on both the zero-shot and one-shot data. In this setting, we exclude the context (the sentences preceding and succeeding the one containing the idioms) and also add the idiom as an additional feature in the “second sentence”.


## Functions for pre-processing

### _get_train_data

This function generates training data in the format required by the huggingface’s example script. It will include and exclude the MWE and the context based on parameters.


In [ ]:
def _get_train_data( data_location, file_name, include_context, include_idiom,language ) :

    file_name = os.path.join( data_location, file_name )

    header, data = load_csv( file_name )

    out_header = [ 'label', 'sentence1' ]
    if include_idiom :
        out_header = [ 'label', 'sentence1', 'sentence2' ]

    # ['DataID', 'Language', 'MWE', 'Setting', 'Previous', 'Target', 'Next', 'Label']
    out_data = list()
    for elem in data :
        label     = elem[ header.index( 'Label'  ) ]
        sentence1 = elem[ header.index( 'Target' ) ]
        if language=="EN":
          if elem[header.index('Language')] != language:
            continue
        if language=="PT":
          if elem[header.index('Language')] != language:
            continue
        if include_context :
            sentence1 = ' '.join( [ elem[ header.index( 'Previous' ) ], elem[ header.index( 'Target' ) ], elem[ header.index( 'Next' ) ] ] )
        this_row = None
        if not include_idiom :
            this_row = [ label, sentence1 ]
        else :
            sentence2 = elem[ header.index( 'MWE' ) ]
            this_row = [ label, sentence1, sentence2 ]
        out_data.append( this_row )
        assert len( out_header ) == len( this_row )
    fin=[out_header] + out_data
    return fin

### _get_dev_eval_data

This function generates training dev and eval data in the format required by the huggingface’s example script. It will include and exclude the MWE and the context based on parameters.

Additionally, if there is no gold label provides (as in the case of eval) it will generate a file that can be used to generate predictions.


In [ ]:
def _get_dev_eval_data( data_location, input_file_name, gold_file_name, include_context, include_idiom ) :

    input_headers, input_data = load_csv( os.path.join( data_location, input_file_name ) )
    gold_header  = gold_data = None
    if not gold_file_name is None :
        gold_header  , gold_data  = load_csv( os.path.join( data_location, gold_file_name  ) )
        assert len( input_data ) == len( gold_data )

    # ['ID', 'Language', 'MWE', 'Previous', 'Target', 'Next']
    # ['ID', 'DataID', 'Language', 'Label']

    out_header = [ 'label', 'sentence1' ]
    if include_idiom :
        out_header = [ 'label', 'sentence1', 'sentence2' ]

    out_data = list()
    for index in range( len( input_data ) ) :
        label = 1
        if not gold_file_name is None :
            this_input_id = input_data[ index ][ input_headers.index( 'ID' ) ]
            this_gold_id  = gold_data [ index ][ gold_header  .index( 'ID' ) ]
            assert this_input_id == this_gold_id

            label     = gold_data[ index ][ gold_header.index( 'Label'  ) ]

        elem      = input_data[ index ]
        sentence1 = elem[ input_headers.index( 'Target' ) ]
        if include_context :
            sentence1 = ' '.join( [ elem[ input_headers.index( 'Previous' ) ], elem[ input_headers.index( 'Target' ) ], elem[ input_headers.index( 'Next' ) ] ] )
        this_row = None
        if not include_idiom :
            this_row = [ label, sentence1 ]
        else :
            sentence2 = elem[ input_headers.index( 'MWE' ) ]
            this_row = [ label, sentence1, sentence2 ]
        assert len( out_header ) == len( this_row )
        out_data.append( this_row )


    return [ out_header ] + out_data


### create_data

This function generates the training, development and evaluation data.


In [ ]:
"""
Based on the results presented in `AStitchInLanguageModels' we work with not including the idiom for the zero shot setting and including it in the one shot setting.
"""
def create_data( input_location, output_location ) :


    ## Zero shot data
    train_data_multi = _get_train_data(
        data_location   = input_location,
        file_name       = 'train_zero_shot.csv',
        include_context = True,
        include_idiom   = False,
        language        = 'Multi'
    )
    write_csv( train_data_multi, os.path.join( output_location, 'ZeroShot', 'train_multi.csv' ) )

    train_data_en = _get_train_data(
        data_location   = input_location,
        file_name       = 'train_zero_shot.csv',
        include_context = True,
        include_idiom   = False,
        language        = 'EN'
    )
    write_csv( train_data_en, os.path.join( output_location, 'ZeroShot', 'train_en.csv' ) )

    train_data_PT = _get_train_data(
        data_location   = input_location,
        file_name       = 'train_zero_shot.csv',
        include_context = True,
        include_idiom   = False,
        language        = 'PT'
    )
    write_csv( train_data_PT, os.path.join( output_location, 'ZeroShot', 'train_pt.csv' ) )

    dev_data = _get_dev_eval_data(
        data_location    = input_location,
        input_file_name  = 'dev.csv',
        gold_file_name   = 'dev_gold.csv',
        include_context  = True,
        include_idiom    = False
    )
    write_csv( dev_data, os.path.join( output_location, 'ZeroShot', 'dev.csv' ) )

    eval_data = _get_dev_eval_data(
        data_location    = input_location,
        input_file_name  = 'eval.csv',
        gold_file_name   = None , ## Don't have gold evaluation file -- submit to CodaLab
        include_context  = True,
        include_idiom    = False
    )
    write_csv( eval_data, os.path.join( output_location, 'ZeroShot', 'eval.csv' ) )


    return

## Setup and Create data

In [ ]:
!ls

AStitchInLanguageModels  models       SemEval_2022_Task2-idiomaticity
Data			 sample_data  transformers


In [ ]:
outpath = 'Data'

Path( os.path.join( outpath, 'ZeroShot' ) ).mkdir(parents=True, exist_ok=True)
Path( os.path.join( outpath, 'OneShot' ) ).mkdir(parents=True, exist_ok=True)

create_data( 'SemEval_2022_Task2-idiomaticity/SubTaskA/Data/', outpath )

Wrote Data/ZeroShot/train_multi.csv
Wrote Data/ZeroShot/train_en.csv
Wrote Data/ZeroShot/train_pt.csv
Wrote Data/ZeroShot/dev.csv
Wrote Data/ZeroShot/eval.csv


In [ ]:
train_multi=pd.read_csv("/content/SemEval_2022_Task2-idiomaticity/SubTaskA/Data/train_zero_shot.csv")
print(train_multi)

                         DataID Language           MWE    Setting  \
0      train_zero_shot.EN.168.1       EN  double dutch  zero_shot   
1      train_zero_shot.EN.168.2       EN  double dutch  zero_shot   
2      train_zero_shot.EN.168.3       EN  double dutch  zero_shot   
3      train_zero_shot.EN.168.4       EN  double dutch  zero_shot   
4      train_zero_shot.EN.168.5       EN  double dutch  zero_shot   
...                         ...      ...           ...        ...   
4486   train_zero_shot.PT.351.8       PT  gato-pingado  zero_shot   
4487   train_zero_shot.PT.351.9       PT  gato-pingado  zero_shot   
4488  train_zero_shot.PT.351.10       PT  gato-pingado  zero_shot   
4489  train_zero_shot.PT.351.11       PT  gato-pingado  zero_shot   
4490  train_zero_shot.PT.351.12       PT  gato-pingado  zero_shot   

                                               Previous  \
0     This inspired others to jump ropes as a leisur...   
1     In the age of chivalry a man paid for the woma.

In [ ]:
train_multi=pd.read_csv("/content/SemEval_2022_Task2-idiomaticity/SubTaskA/Data/train_zero_shot.csv")
#train_multi_df=pd.DataFrame(train_multi)
X=train_multi.drop('Language',axis=1)
y=train_multi['Language']
under=RandomUnderSampler(sampling_strategy='majority')
X_over, y_over = under.fit_resample(X, y)
train_multi_undersampled = pd.concat([X_over, y_over], axis=1, join='inner')
train_en_undersampled=train_multi_undersampled[train_multi_undersampled['Language']=='EN']
filepath=''
train_en_undersampled.to_csv('Data/ZeroShot/train_en_undersampled.csv')
train_data_multi = _get_train_data(
        data_location   = 'Data/ZeroShot/',
        file_name       = 'train_en_undersampled.csv',
        include_context = True,
        include_idiom   = False,
        language        = 'Multi'
    )
write_csv( train_data_multi, os.path.join( 'Data', 'ZeroShot', 'train_undersampled_final.csv' ) )

Wrote Data/ZeroShot/train_undersampled_final.csv


In [ ]:
train_multi=pd.read_csv("/content/SemEval_2022_Task2-idiomaticity/SubTaskA/Data/train_zero_shot.csv")
#train_multi_df=pd.DataFrame(train_multi)
X=train_multi.drop('Language',axis=1)
y=train_multi['Language']
under=RandomUnderSampler(sampling_strategy='majority')
X_over, y_over = under.fit_resample(X, y)
train_multi_undersampled = pd.concat([X_over, y_over], axis=1, join='inner')
filepath=''
train_en_undersampled.to_csv('Data/ZeroShot/train_en_undersampled.csv')
train_data_multi = _get_train_data(
        data_location   = 'Data/ZeroShot/',
        file_name       = 'train_en_undersampled.csv',
        include_context = True,
        include_idiom   = False,
        language        = 'Multi'
    )
write_csv( train_data_multi, os.path.join( 'Data', 'ZeroShot', 'train_undersampled_multi_final.csv' ) )

Wrote Data/ZeroShot/train_undersampled_multi_final.csv


# *MFC class*

In [ ]:
import sklearn
import pandas as pd
from sklearn.dummy import DummyClassifier
path_input="Data/ZeroShot/train_multi.csv"
path_test_multi= '/content/SemEval_2022_Task2-idiomaticity/SubTaskA/Data/dev_gold.csv'
df_train=pd.read_csv(path_input)
df_test=pd.read_csv(path_test_multi)
dummy_clf=DummyClassifier(strategy="most_frequent")
dummy_clf.fit(df_train['sentence1'],df_train['label'])
print("This is the MFC for the trained on multilingual data and tested on multilingual data")
print(dummy_clf.score(df_test['Language'],df_test['Label']))

This is the MFC for the trained on multilingual data and tested on multilingual data
0.4546684709066306


In [ ]:
import sklearn
import pandas as pd
from sklearn.dummy import DummyClassifier
path_input="Data/ZeroShot/train_en.csv"
path_test= '/content/SemEval_2022_Task2-idiomaticity/SubTaskA/Data/dev_gold.csv'
df_train=pd.read_csv(path_input)
df_test=pd.read_csv(path_test)
print(df_train)
dummy_clf=DummyClassifier(strategy="most_frequent")
dummy_clf.fit(df_train['sentence1'],df_train['label'])
print("This is the MFC for the trained on multilingual data and multilingual data")
print(dummy_clf.score(df_test['Language'],df_test['Label']))

      label                                          sentence1
0         0  This inspired others to jump ropes as a leisur...
1         0  In the age of chivalry a man paid for the woma...
2         0  To her eternal credit, she kept both India and...
3         0  While pharmaceutical companies were researchin...
4         0  Coronavirus in Europe   * Brexit   * Brussels ...
...     ...                                                ...
3322      0  He has recorded 95 tackles and just 6 sacks si...
3323      0  But when the regulars started to return, Ighal...
3324      0  My job as a head coach is to make the team as ...
3325      0  Golden paper wasps have demanding social lives...
3326      0  If only one is selected, mock drafters seem to...

[3327 rows x 2 columns]
This is the MFC for the trained on multilingual data and multilingual data
0.4546684709066306


In [ ]:
import sklearn
import pandas as pd
from sklearn.dummy import DummyClassifier
path_input="Data/ZeroShot/train_pt.csv"
path_test= '/content/SemEval_2022_Task2-idiomaticity/SubTaskA/Data/dev_gold.csv'
df_train=pd.read_csv(path_input)
df_test=pd.read_csv(path_test)
print(df_train)
dummy_clf=DummyClassifier(strategy="most_frequent")
dummy_clf.fit(df_train['sentence1'],df_train['label'])
print("This is the MFC for the portguese setup and tested on multilingual data")
print(dummy_clf.score(df_test['Language'],df_test['Label']))

      label                                          sentence1
0         1  O presidente Teodoro Obiang Nguema, chefe de E...
1         1  Explosões deixam 15 mortos e 500 feridos na Gu...
2         1  O protagonista Alexander Pechersky vai liderar...
3         1  Às vesperas das eleiçoes, milhares de mulheres...
4         1  O presidente Teodoro Obiang Nguema, chefe de E...
...     ...                                                ...
1159      0  Era o ano de 1968.  A estação de passageiros t...
1160      0  De acordo com informações vazadas de dentro da...
1161      0  O radialista Roberto Toledo apresentou o show ...
1162      1  O carnaval em Goiânia também tem espaço para o...
1163      1  A concentração aconteceu na Avenida Circular, ...

[1164 rows x 2 columns]
This is the MFC for the portguese setup and tested on multilingual data
0.4546684709066306


In [ ]:
import sklearn
import pandas as pd
from sklearn.dummy import DummyClassifier
path_input="Data/ZeroShot/train_multi.csv"
path_test= '/content/SemEval_2022_Task2-idiomaticity/SubTaskA/Data/dev_gold.csv'
df_train=pd.read_csv(path_input)
df_test=pd.read_csv(path_test)
df_test=df_test[df_test['Language']=='EN']
print(df_train)
dummy_clf=DummyClassifier(strategy="most_frequent")
dummy_clf.fit(df_train['sentence1'],df_train['label'])
print("This is the MFC for the portguese setup and tested on multilingual data")
print(dummy_clf.score(df_test['Language'],df_test['Label']))

      label                                          sentence1
0         0  This inspired others to jump ropes as a leisur...
1         0  In the age of chivalry a man paid for the woma...
2         0  To her eternal credit, she kept both India and...
3         0  While pharmaceutical companies were researchin...
4         0  Coronavirus in Europe   * Brexit   * Brussels ...
...     ...                                                ...
4486      0  Era o ano de 1968.  A estação de passageiros t...
4487      0  De acordo com informações vazadas de dentro da...
4488      0  O radialista Roberto Toledo apresentou o show ...
4489      1  O carnaval em Goiânia também tem espaço para o...
4490      1  A concentração aconteceu na Avenida Circular, ...

[4491 rows x 2 columns]
This is the MFC for the portguese setup and tested on multilingual data
0.3905579399141631


In [ ]:
import sklearn
import pandas as pd
from sklearn.dummy import DummyClassifier
path_input="Data/ZeroShot/train_en.csv"
path_test= '/content/SemEval_2022_Task2-idiomaticity/SubTaskA/Data/dev_gold.csv'
df_train=pd.read_csv(path_input)
df_test=pd.read_csv(path_test)
df_test=df_test[df_test['Language']=='EN']
print(df_train)
dummy_clf=DummyClassifier(strategy="most_frequent")
dummy_clf.fit(df_train['sentence1'],df_train['label'])
print("This is the MFC for the portguese setup and tested on multilingual data")
print(dummy_clf.score(df_test['Language'],df_test['Label']))

      label                                          sentence1
0         0  This inspired others to jump ropes as a leisur...
1         0  In the age of chivalry a man paid for the woma...
2         0  To her eternal credit, she kept both India and...
3         0  While pharmaceutical companies were researchin...
4         0  Coronavirus in Europe   * Brexit   * Brussels ...
...     ...                                                ...
3322      0  He has recorded 95 tackles and just 6 sacks si...
3323      0  But when the regulars started to return, Ighal...
3324      0  My job as a head coach is to make the team as ...
3325      0  Golden paper wasps have demanding social lives...
3326      0  If only one is selected, mock drafters seem to...

[3327 rows x 2 columns]
This is the MFC for the portguese setup and tested on multilingual data
0.3905579399141631


In [ ]:
import sklearn
import pandas as pd
from sklearn.dummy import DummyClassifier
path_input="Data/ZeroShot/train_pt.csv"
path_test= '/content/SemEval_2022_Task2-idiomaticity/SubTaskA/Data/dev_gold.csv'
df_train=pd.read_csv(path_input)
df_test=pd.read_csv(path_test)
df_test=df_test[df_test['Language']=='EN']
print(df_train)
dummy_clf=DummyClassifier(strategy="most_frequent")
dummy_clf.fit(df_train['sentence1'],df_train['label'])
print("This is the MFC for the portguese setup and tested on english data")
print(dummy_clf.score(df_test['Language'],df_test['Label']))

      label                                          sentence1
0         1  O presidente Teodoro Obiang Nguema, chefe de E...
1         1  Explosões deixam 15 mortos e 500 feridos na Gu...
2         1  O protagonista Alexander Pechersky vai liderar...
3         1  Às vesperas das eleiçoes, milhares de mulheres...
4         1  O presidente Teodoro Obiang Nguema, chefe de E...
...     ...                                                ...
1159      0  Era o ano de 1968.  A estação de passageiros t...
1160      0  De acordo com informações vazadas de dentro da...
1161      0  O radialista Roberto Toledo apresentou o show ...
1162      1  O carnaval em Goiânia também tem espaço para o...
1163      1  A concentração aconteceu na Avenida Circular, ...

[1164 rows x 2 columns]
This is the MFC for the portguese setup and tested on english data
0.3905579399141631


In [ ]:
import sklearn
import pandas as pd
from sklearn.dummy import DummyClassifier
path_input="Data/ZeroShot/train_multi.csv"
path_test= '/content/SemEval_2022_Task2-idiomaticity/SubTaskA/Data/dev_gold.csv'
df_train=pd.read_csv(path_input)
df_test=pd.read_csv(path_test)
df_test=df_test[df_test['Language']=='PT']
print(df_train)
dummy_clf=DummyClassifier(strategy="most_frequent")
dummy_clf.fit(df_train['sentence1'],df_train['label'])
print("This is the MFC for the portguese setup and tested on portuguese data")
print(dummy_clf.score(df_test['Language'],df_test['Label']))

      label                                          sentence1
0         0  This inspired others to jump ropes as a leisur...
1         0  In the age of chivalry a man paid for the woma...
2         0  To her eternal credit, she kept both India and...
3         0  While pharmaceutical companies were researchin...
4         0  Coronavirus in Europe   * Brexit   * Brussels ...
...     ...                                                ...
4486      0  Era o ano de 1968.  A estação de passageiros t...
4487      0  De acordo com informações vazadas de dentro da...
4488      0  O radialista Roberto Toledo apresentou o show ...
4489      1  O carnaval em Goiânia também tem espaço para o...
4490      1  A concentração aconteceu na Avenida Circular, ...

[4491 rows x 2 columns]
This is the MFC for the portguese setup and tested on portuguese data
0.5641025641025641


In [ ]:
import sklearn
import pandas as pd
from sklearn.dummy import DummyClassifier
path_input="Data/ZeroShot/train_en.csv"
path_test= '/content/SemEval_2022_Task2-idiomaticity/SubTaskA/Data/dev_gold.csv'
df_train=pd.read_csv(path_input)
df_test=pd.read_csv(path_test)
df_test=df_test[df_test['Language']=='PT']
print(df_train)
dummy_clf=DummyClassifier(strategy="most_frequent")
dummy_clf.fit(df_train['sentence1'],df_train['label'])
print("This is the MFC for the english setup and tested on portuguese data")
print(dummy_clf.score(df_test['Language'],df_test['Label']))

      label                                          sentence1
0         0  This inspired others to jump ropes as a leisur...
1         0  In the age of chivalry a man paid for the woma...
2         0  To her eternal credit, she kept both India and...
3         0  While pharmaceutical companies were researchin...
4         0  Coronavirus in Europe   * Brexit   * Brussels ...
...     ...                                                ...
3322      0  He has recorded 95 tackles and just 6 sacks si...
3323      0  But when the regulars started to return, Ighal...
3324      0  My job as a head coach is to make the team as ...
3325      0  Golden paper wasps have demanding social lives...
3326      0  If only one is selected, mock drafters seem to...

[3327 rows x 2 columns]
This is the MFC for the english setup and tested on portuguese data
0.5641025641025641


# Zero Shot Setting

## Train Zero shot

In [ ]:
!python /content/AStitchInLanguageModels/Dataset/Task2/Utils/run_glue_f1_macro.py \
    	--model_name_or_path 'xlm-roberta-base' \
    	--do_train \
    	--do_eval \
    	--max_seq_length 128 \
    	--per_device_train_batch_size 32 \
    	--learning_rate 2e-5 \
    	--num_train_epochs 9 \
    	--evaluation_strategy "epoch" \
    	--output_dir models/ZeroShot/0/ \
    	--seed 0 \
    	--train_file      Data/ZeroShot/train_undersampled_multi_final.csv \
    	--validation_file Data/ZeroShot/dev.csv \
	    --evaluation_strategy "epoch" \
	    --save_strategy "epoch"  \
	    --load_best_model_at_end \
	    --metric_for_best_model "accuracy" \
	    --save_total_limit 1

2023-03-24 19:00:07.015640: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-24 19:00:07.015807: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-24 19:00:07.015835: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
03/24/2023 19:00:09 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
03/24/2023 19:00:09 - INFO 

In [ ]:
!python /content/AStitchInLanguageModels/Dataset/Task2/Utils/run_glue_f1_macro.py \
    	--model_name_or_path 'xlm-roberta-base' \
    	--do_train \
    	--do_eval \
    	--max_seq_length 128 \
    	--per_device_train_batch_size 32 \
    	--learning_rate 2e-5 \
    	--num_train_epochs 9 \
    	--evaluation_strategy "epoch" \
    	--output_dir models/ZeroShot/11/ \
    	--seed 0 \
    	--train_file      Data/ZeroShot/train_en.csv \
    	--validation_file Data/ZeroShot/dev.csv \
	    --evaluation_strategy "epoch" \
	    --save_strategy "epoch"  \
	    --load_best_model_at_end \
	    --metric_for_best_model "accuracy" \
	    --save_total_limit 1

2023-03-24 18:55:44.348191: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-24 18:55:44.348344: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-24 18:55:44.348366: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
03/24/2023 18:55:47 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
03/24/2023 18:55:47 - INFO 

In [ ]:
!python /content/AStitchInLanguageModels/Dataset/Task2/Utils/run_glue_f1_macro.py \
    	--model_name_or_path 'xlm-roberta-base' \
    	--do_train \
    	--do_eval \
    	--max_seq_length 128 \
    	--per_device_train_batch_size 32 \
    	--learning_rate 2e-5 \
    	--num_train_epochs 9 \
    	--evaluation_strategy "epoch" \
    	--output_dir models/ZeroShot/2/ \
    	--seed 0 \
    	--train_file      Data/ZeroShot/train_pt.csv \
    	--validation_file Data/ZeroShot/dev.csv \
	    --evaluation_strategy "epoch" \
	    --save_strategy "epoch"  \
	    --load_best_model_at_end \
	    --metric_for_best_model "accuracy" \
	    --save_total_limit 1

In [ ]:
!python /content/AStitchInLanguageModels/Dataset/Task2/Utils/run_glue_f1_macro.py \
    	--model_name_or_path 'xlm-roberta-base' \
    	--do_train \
    	--do_eval \
    	--max_seq_length 128 \
    	--per_device_train_batch_size 32 \
    	--learning_rate 2e-5 \
    	--num_train_epochs 9 \
    	--evaluation_strategy "epoch" \
    	--output_dir models/ZeroShot/6/ \
    	--seed 0 \
    	--train_file      Data/ZeroShot/train_undersampled_final.csv \
    	--validation_file Data/ZeroShot/dev.csv \
	    --evaluation_strategy "epoch" \
	    --save_strategy "epoch"  \
	    --load_best_model_at_end \
	    --metric_for_best_model "accuracy" \
	    --save_total_limit 1


In [ ]:
!python /content/AStitchInLanguageModels/Dataset/Task2/Utils/run_glue_f1_macro.py \
    	--model_name_or_path 'xlm-roberta-base' \
    	--do_train \
    	--do_eval \
    	--max_seq_length 128 \
    	--per_device_train_batch_size 32 \
    	--learning_rate 2e-5 \
    	--num_train_epochs 9 \
    	--evaluation_strategy "epoch" \
    	--output_dir models/ZeroShot/5/ \
    	--seed 0 \
    	--train_file      Data/ZeroShot/train_undersampled_multi_final.csv \
    	--validation_file Data/ZeroShot/dev.csv \
	    --evaluation_strategy "epoch" \
	    --save_strategy "epoch"  \
	    --load_best_model_at_end \
	    --metric_for_best_model "accuracy" \
	    --save_total_limit 1

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
## Create save path
!mkdir -p /content/gdrive/MyDrive/ColabData/SemEval2022Task2/TaskA/ZeroShot/0/
## Copy saved model.
!cp -r /content/models/ZeroShot/0/* /content/gdrive/MyDrive/ColabData/SemEval2022Task2/TaskA/ZeroShot/0/

In [ ]:
!mkdir -p /content/gdrive/MyDrive/ColabData/SemEval2022Task2/TaskA/ZeroShot/11/
## Copy saved model.
!cp -r /content/models/ZeroShot/11/* /content/gdrive/MyDrive/ColabData/SemEval2022Task2/TaskA/ZeroShot/11/

In [ ]:
!mkdir -p /content/gdrive/MyDrive/ColabData/SemEval2022Task2/TaskA/ZeroShot/2/
## Copy saved model.
!cp -r /content/models/ZeroShot/2/* /content/gdrive/MyDrive/ColabData/SemEval2022Task2/TaskA/ZeroShot/2/

In [ ]:
!mkdir -p /content/gdrive/MyDrive/ColabData/SemEval2022Task2/TaskA/ZeroShot/6/
## Copy saved model.
!cp -r /content/models/ZeroShot/6/* /content/gdrive/MyDrive/ColabData/SemEval2022Task2/TaskA/ZeroShot/6/

In [ ]:
!mkdir -p /content/gdrive/MyDrive/ColabData/SemEval2022Task2/TaskA/ZeroShot/5/
## Copy saved model.
!cp -r /content/models/ZeroShot/5/* /content/gdrive/MyDrive/ColabData/SemEval2022Task2/TaskA/ZeroShot/5/

In [ ]:
!mkdir -p /content/gdrive/MyDrive/SemEval_2022_Task2-idiomaticity
## Copy saved model.
!cp -r SemEval_2022_Task2-idiomaticity/* /content/gdrive/MyDrive/SemEval_2022_Task2-idiomaticity

In [ ]:
## Bring back saved model here.
#!mkdir -p /content/models/ZeroShot/0/
# !cp -r /content/gdrive/MyDrive/ColabData/SemEval2022Task2/TaskA/ZeroShot/0/* /content/models/ZeroShot/0/

## Evaluation On Dev Data

In [ ]:
!python /content/AStitchInLanguageModels/Dataset/Task2/Utils/run_glue_f1_macro.py \
    	--model_name_or_path '/content/gdrive/MyDrive/ColabData/SemEval2022Task2/TaskA/ZeroShot/0/' \
    	--do_predict \
    	--max_seq_length 128 \
    	--per_device_train_batch_size 32 \
    	--learning_rate 2e-5 \
    	--num_train_epochs 9 \
    	--evaluation_strategy "epoch" \
    	--output_dir models/ZeroShot/0/eval-dev/ \
    	--seed 0 \
    	--train_file      Data/ZeroShot/train_multi.csv \
    	--validation_file Data/ZeroShot/dev.csv \
      --test_file Data/ZeroShot/dev.csv \
	    --evaluation_strategy "epoch" \
	    --save_strategy "epoch"  \
	    --load_best_model_at_end \
	    --metric_for_best_model "f1" \
	    --save_total_limit 1

In [ ]:
!python /content/AStitchInLanguageModels/Dataset/Task2/Utils/run_glue_f1_macro.py \
    	--model_name_or_path '/content/gdrive/MyDrive/ColabData/SemEval2022Task2/TaskA/ZeroShot/11/' \
    	--do_predict \
    	--max_seq_length 128 \
    	--per_device_train_batch_size 32 \
    	--learning_rate 2e-5 \
    	--num_train_epochs 9 \
    	--evaluation_strategy "epoch" \
    	--output_dir models/ZeroShot/11/eval-dev/ \
    	--seed 0 \
    	--train_file      Data/ZeroShot/train_en.csv \
    	--validation_file Data/ZeroShot/dev.csv \
      --test_file Data/ZeroShot/dev.csv \
	    --evaluation_strategy "epoch" \
	    --save_strategy "epoch"  \
	    --load_best_model_at_end \
	    --metric_for_best_model "f1" \
	    --save_total_limit 1

In [ ]:
!python /content/AStitchInLanguageModels/Dataset/Task2/Utils/run_glue_f1_macro.py \
    	--model_name_or_path '/content/gdrive/MyDrive/ColabData/SemEval2022Task2/TaskA/ZeroShot/2/' \
    	--do_predict \
    	--max_seq_length 128 \
    	--per_device_train_batch_size 32 \
    	--learning_rate 2e-5 \
    	--num_train_epochs 9 \
    	--evaluation_strategy "epoch" \
    	--output_dir models/ZeroShot/2/eval-dev/ \
    	--seed 0 \
    	--train_file      Data/ZeroShot/train_pt.csv \
    	--validation_file Data/ZeroShot/dev.csv \
      --test_file Data/ZeroShot/dev.csv \
	    --evaluation_strategy "epoch" \
	    --save_strategy "epoch"  \
	    --load_best_model_at_end \
	    --metric_for_best_model "f1" \
	    --save_total_limit 1

In [ ]:
!python /content/AStitchInLanguageModels/Dataset/Task2/Utils/run_glue_f1_macro.py \
    	--model_name_or_path '/content/gdrive/MyDrive/ColabData/SemEval2022Task2/TaskA/ZeroShot/6/' \
    	--do_predict \
    	--max_seq_length 128 \
    	--per_device_train_batch_size 32 \
    	--learning_rate 2e-5 \
    	--num_train_epochs 9 \
    	--evaluation_strategy "epoch" \
    	--output_dir models/ZeroShot/6/eval-dev/ \
    	--seed 0 \
    	--train_file      Data/ZeroShot/train_undersampled_final.csv \
    	--validation_file Data/ZeroShot/dev.csv \
      --test_file Data/ZeroShot/dev.csv \
	    --evaluation_strategy "epoch" \
	    --save_strategy "epoch"  \
	    --load_best_model_at_end \
	    --metric_for_best_model "f1" \
	    --save_total_limit 1

In [ ]:
!python /content/AStitchInLanguageModels/Dataset/Task2/Utils/run_glue_f1_macro.py \
    	--model_name_or_path '/content/gdrive/MyDrive/ColabData/SemEval2022Task2/TaskA/ZeroShot/5/' \
    	--do_predict \
    	--max_seq_length 128 \
    	--per_device_train_batch_size 32 \
    	--learning_rate 2e-5 \
    	--num_train_epochs 9 \
    	--evaluation_strategy "epoch" \
    	--output_dir models/ZeroShot/5/eval-dev/ \
    	--seed 0 \
    	--train_file      Data/ZeroShot/train_undersampled_multi_final.csv \
    	--validation_file Data/ZeroShot/dev.csv \
      --test_file Data/ZeroShot/dev.csv \
	    --evaluation_strategy "epoch" \
	    --save_strategy "epoch"  \
	    --load_best_model_at_end \
	    --metric_for_best_model "f1" \
	    --save_total_limit 1

### Use predictions to create the submission file (for dev data)

In [ ]:
params_multi = {
    'submission_format_file' : '/content/SemEval_2022_Task2-idiomaticity/SubTaskA/Data/dev_submission_format.csv' ,
    'input_file'             : '/content/SemEval_2022_Task2-idiomaticity/SubTaskA/Data/dev.csv'                   ,
    'prediction_format_file' : '/content/models/ZeroShot/0/eval-dev/test_results_None.txt'                        ,
    }
params_multi[ 'setting' ] = 'zero_shot'
params_en = {
    'submission_format_file' : '/content/SemEval_2022_Task2-idiomaticity/SubTaskA/Data/dev_submission_format.csv' ,
    'input_file'             : '/content/SemEval_2022_Task2-idiomaticity/SubTaskA/Data/dev.csv'                   ,
    'prediction_format_file' : '/content/models/ZeroShot/11/eval-dev/test_results_None.txt'                        ,
    }
params_en[ 'setting' ] = 'zero_shot'
params_pt = {
    'submission_format_file' : '/content/SemEval_2022_Task2-idiomaticity/SubTaskA/Data/dev_submission_format.csv' ,
    'input_file'             : '/content/SemEval_2022_Task2-idiomaticity/SubTaskA/Data/dev.csv'                   ,
    'prediction_format_file' : '/content/models/ZeroShot/2/eval-dev/test_results_None.txt'                        ,
    }
params_pt[ 'setting' ] = 'zero_shot'
params_undersampled = {
    'submission_format_file' : '/content/SemEval_2022_Task2-idiomaticity/SubTaskA/Data/dev_submission_format.csv' ,
    'input_file'             : '/content/SemEval_2022_Task2-idiomaticity/SubTaskA/Data/dev.csv'                   ,
    'prediction_format_file' : '/content/models/ZeroShot/6/eval-dev/test_results_None.txt'                        ,
    }
params_undersampled[ 'setting' ] = 'zero_shot'
params_undersampled_multi = {
    'submission_format_file' : '/content/SemEval_2022_Task2-idiomaticity/SubTaskA/Data/dev_submission_format.csv' ,
    'input_file'             : '/content/SemEval_2022_Task2-idiomaticity/SubTaskA/Data/dev.csv'                   ,
    'prediction_format_file' : '/content/models/ZeroShot/5/eval-dev/test_results_None.txt'                        ,
    }
params_undersampled_multi[ 'setting' ] = 'zero_shot'

In [ ]:
updated_data_multi = insert_to_submission_file( **params_multi )
updated_data_en = insert_to_submission_file( **params_en )
updated_data_pt = insert_to_submission_file( **params_pt )
updated_data_undersampled = insert_to_submission_file( **params_undersampled )
updated_data_undersampled_multi = insert_to_submission_file( **params_undersampled_multi )

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!mkdir -p outputs

In [ ]:
write_csv( updated_data_multi, 'outputs/zero_shot_dev_formated_multi.csv' )
write_csv( updated_data_en, 'outputs/zero_shot_dev_formated_en.csv' )
write_csv( updated_data_pt, 'outputs/zero_shot_dev_formated_pt.csv' )
write_csv( updated_data_undersampled, 'outputs/zero_shot_dev_formated_undersampled.csv' )
write_csv( updated_data_undersampled_multi, 'outputs/zero_shot_dev_formated_undersampled_multi.csv' )